In [1]:
import tensorflow as tf
#import tensorflow_hub as hub
import requests
print(tf.__version__)
import zipfile
import requests
import os
import time
import pandas as pd
import random
import shutil
import tempfile
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
import numpy as np
from PIL import Image
import pickle
from tensorflow.keras.models import load_model, Model
from PIL import Image
from PIL.PngImagePlugin import PngImageFile
import matplotlib.pyplot as plt
import glob
from functools import partial
import nltk
import absl

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        print("Couldn't set memory_growth")
        pass
    
    
def fix_random_seed(seed):
    """ Setting the random seed of various libraries """
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
random_seed=4321
fix_random_seed(random_seed)

2.4.1


http://archive.ics.uci.edu/ml/datasets/Forest+Fires

In [2]:
# Downloading the data
# http://www.thespermwhale.com/jaseweston/babi/CBTest.tgz

import os
import requests
import tarfile

import shutil

# Retrieve the data
if not os.path.exists(os.path.join('data', 'csv', 'forestfires.csv')):
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists(os.path.join('data', 'csv')):
        os.makedirs(os.path.join('data', 'csv'))
    
    # Write to a file
    with open(os.path.join('data', 'csv', 'forestfires.csv'), 'wb') as f:
        f.write(r.content)
        
else:
    print("The forestfires.csv file already exists.")
    
    
if not os.path.exists(os.path.join('data', 'forestfires.names')):
    
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.names"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists('data'):
        os.makedirs('data')
    
    # Write to a file
    with open(os.path.join('data', 'forestfires.names'), 'wb') as f:
        f.write(r.content)
        
else:
    print("The forestfires.names file already exists.")



The forestfires.csv file already exists.
The forestfires.names file already exists.


In [3]:
# This is the directory containing the TFX Chicago Taxi Pipeline example.
_pipeline_root = os.path.join('pipeline', 'examples', 'forest_fires_pipeline')

# This is the path where your model will be pushed for serving.
# tempfile.mkdtemp() creates a temporary directory (e.g. tmpxxxxxx) in default OS temporary directory
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model', 'forest_fires')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)


In [4]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
# pipeline_name = "forest_fires", pipeline_root=_pipeline_root
context = InteractiveContext(pipeline_name = "forest_fires", pipeline_root=_pipeline_root)

INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'


## Had to install apache-beam 2.29 (not 2.30) coz 2.30 was giving the error
Attribute error: type StringMethods does not have attribute fullmatch

In [5]:
from tfx.components import CsvExampleGen

example_gen = CsvExampleGen(input_base=os.path.join('data', 'csv'))
context.run(example_gen)

INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 19
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 9
        type_id: 5
        uri: "pipeline/examples/forest_fires_pipeline/CsvExampleGen/examples/9"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:25478,xor_checksum:1623578318,sum_checksum:1623578318"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1623578384841
        last_update_time_since_epoch: 1623578385895
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [6]:
print(example_gen.outputs)

{'examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 9
type_id: 5
uri: "pipeline/examples/forest_fires_pipeline/CsvExampleGen/examples/9"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:25478,xor_checksum:1623578318,sum_checksum:1623578318"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    string_value: "0"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "0.29.0"
  }
}
state: LIVE
create_time_since_epoch: 1623578384841
last_update_time_since_epoch: 1623578385895
, artifact_type: id: 5
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STR

In [7]:
!ls -lah /tmp/tfx-interactive-2021-06-11T07_06_53.060198-4jyvtunk/CsvExampleGen/examples/3
!ls -lah /tmp/tfx-interactive-2021-06-11T07_06_53.060198-4jyvtunk/CsvExampleGen/examples/3/Split-eval

ls: cannot access '/tmp/tfx-interactive-2021-06-11T07_06_53.060198-4jyvtunk/CsvExampleGen/examples/3': No such file or directory
ls: cannot access '/tmp/tfx-interactive-2021-06-11T07_06_53.060198-4jyvtunk/CsvExampleGen/examples/3/Split-eval': No such file or directory


In [8]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)


["train", "eval"] pipeline/examples/forest_fires_pipeline/CsvExampleGen/examples/9


## Look at training examples

In [9]:
import pprint
pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)


features {
  feature {
    key: "DC"
    value {
      float_list {
        value: 669.0999755859375
      }
    }
  }
  feature {
    key: "DMC"
    value {
      float_list {
        value: 35.400001525878906
      }
    }
  }
  feature {
    key: "FFMC"
    value {
      float_list {
        value: 90.5999984741211
      }
    }
  }
  feature {
    key: "ISI"
    value {
      float_list {
        value: 6.699999809265137
      }
    }
  }
  feature {
    key: "RH"
    value {
      int64_list {
        value: 33
      }
    }
  }
  feature {
    key: "X"
    value {
      int64_list {
        value: 7
      }
    }
  }
  feature {
    key: "Y"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "area"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "day"
    value {
      bytes_list {
        value: "tue"
      }
    }
  }
  feature {
    key: "month"
    value {
      bytes_list {
        value: "oct"
    

## Generate schema

In [10]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 20
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 10
        type_id: 7
        uri: "pipeline/examples/forest_fires_pipeline/StatisticsGen/statistics/10"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1623578435510
        last_update_time_since_epoch: 1623578437413
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [11]:
context.show(statistics_gen.outputs['statistics'])

In [12]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 21
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 11
        type_id: 9
        uri: "pipeline/examples/forest_fires_pipeline/SchemaGen/schema/11"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1623578476753
        last_update_time_since_epoch: 1623578476774
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'day',STRING,required,single,'day'
'month',STRING,required,single,'month'
'DC',FLOAT,required,single,-
'DMC',FLOAT,required,single,-
'FFMC',FLOAT,required,single,-
'ISI',FLOAT,required,single,-
'RH',INT,required,single,-
'X',INT,required,single,-
'Y',INT,required,single,-


https://www.tensorflow.org/tfx/tutorials/tfx/components

In [14]:
%%writefile forest_fires_constants.py

# Categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [7, 12]

CATEGORICAL_FEATURE_KEYS = [
    'day', 'month'
]

DENSE_FLOAT_FEATURE_KEYS = ['DC', 'DMC', 'FFMC', 'ISI', 'rain', 'temp', 'wind', 'X', 'Y']

# Number of buckets used by tf.transform for encoding each feature.
BUCKET_FEATURE_BOUNDARIES = [(33, 66)]
BUCKET_FEATURE_KEYS = ['RH']

VOCAB_FEATURE_KEYS = ['day','month']
VOCAB_FEATURE_MAP_KEYS = [
    ('mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun'), 
    ('jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec')]
# Keys
LABEL_KEY = 'area'

def transformed_name(key):
  return key + '_xf'


Overwriting forest_fires_constants.py


In [25]:
%%writefile forest_fires_transform.py

import tensorflow as tf
import tensorflow_transform as tft

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    # Preserve this feature as a dense float, setting nan's to the mean.
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        sparse_to_dense(inputs[key])
    )

  # Treating categorical features  
  for key in _VOCAB_FEATURE_KEYS:
    # Build a vocabulary for this feature.
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
        sparse_to_dense(inputs[key]),
        num_oov_buckets=1)

    
  for key, boundary in zip(_BUCKET_FEATURE_KEYS, _BUCKET_FEATURE_BOUNDARIES):
    outputs[_transformed_name(key)] = tft.apply_buckets(
        sparse_to_dense(inputs[key]), bucket_boundaries=[boundary])


  outputs[_transformed_name(_LABEL_KEY)] = sparse_to_dense(inputs[_LABEL_KEY])

  return outputs

def sparse_to_dense(x):
    return tf.squeeze(
        tf.sparse.to_dense(tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1])),
        axis=1
    )


Overwriting forest_fires_transform.py


In [26]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('forest_fires_transform.py'))

context.run(transform)


INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Analyze the 'train' split and transform all splits when splits_config is not set.
INFO:absl:/home/thushv89/code/manning_tf2_in_action/Ch14/forest_fires_transform.py is already loaded, reloading
INFO:absl:/home/thushv89/code/manning_tf2_in_action/Ch14/forest_fires_transform.py is already loaded, reloading
INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feat

INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


...

value: "\n\013\n\tConst_5:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"



value: "\n\013\n\tConst_5:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 25
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 41
        type_id: 11
        uri: "pipeline/examples/forest_fires_pipeline/Transform/transform_graph/25"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 42
        type_id: 5
        uri: "pipeline/examples/forest_fires_pipeline/Transform/transformed_examples/25"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 43
        type_id: 12
        uri: "pipeline/examples/forest_fires_pipeline/Transform/updated_analyzer_cache/25"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [27]:
print(transform)

Transform(spec: <tfx.types.standard_component_specs.TransformSpec object at 0x7f660c33c358>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7f660c33c2b0>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Transform, inputs: {'examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 9
type_id: 5
uri: "pipeline/examples/forest_fires_pipeline/CsvExampleGen/examples/9"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:25478,xor_checksum:1623578318,sum_checksum:1623578318"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    string_value: "0"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_pro

In [28]:
import pprint
pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  print(example)
  print("="*50)

features {
  feature {
    key: "DC_xf"
    value {
      float_list {
        value: 0.45569995045661926
      }
    }
  }
  feature {
    key: "DMC_xf"
    value {
      float_list {
        value: -1.2202951908111572
      }
    }
  }
  feature {
    key: "FFMC_xf"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "ISI_xf"
    value {
      float_list {
        value: -0.4968927800655365
      }
    }
  }
  feature {
    key: "RH_xf"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "X_xf"
    value {
      float_list {
        value: 1.0199579000473022
      }
    }
  }
  feature {
    key: "Y_xf"
    value {
      float_list {
        value: -0.24281489849090576
      }
    }
  }
  feature {
    key: "area_xf"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "day_xf"
    value {
      int64_list {
        value: 5
      }
    }
  }
  feature {
    key: "month_xf"
   

In [ ]:
_forest_fires_module_file = 'forest_fires_trainer.py'



In [33]:
%%writefile forest_fires_trainer.py

import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils
from tfx_bsl.tfxio import dataset_options

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_MAX_CATEGORICAL_FEATURE_VALUES = forest_fires_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


# Tf.Transform considers these features as "raw"
def _get_raw_feature_spec(schema):
  return schema_utils.schema_as_feature_spec(schema).feature_spec


def _build_estimator(config, hidden_units):
  """Build an estimator for predicting the tipping behavior of taxi riders.
  Args:
    config: tf.estimator.RunConfig defining the runtime environment for the
      estimator (including model_dir).
    hidden_units: [int], the layer sizes of the DNN (input layer first)
    warm_start_from: Optional directory to warm start from.
  Returns:
    A dict of the following:

      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  
  categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=len(boundaries)+1
      ) for key, boundaries in zip(
          _transformed_names(_VOCAB_FEATURE_KEYS), _BUCKET_FEATURE_BOUNDARIES
      )
  ]
    
  categorical_columns = [
      tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
          key,
          num_buckets=num_buckets,
          default_value=0) for key, num_buckets in zip(
              _transformed_names(_VOCAB_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES)
  ]

  return tf.estimator.DNNLinearCombinedRegressor(
      config=config,
      linear_feature_columns=categorical_columns,
      dnn_feature_columns=real_valued_columns,
      dnn_hidden_units=hidden_units
      )


def _example_serving_receiver_fn(tf_transform_graph, schema):
  """Build the serving in inputs.
  Args:
    tf_transform_graph: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    Tensorflow graph which parses examples, applying tf-transform to them.
  """
  raw_feature_spec = _get_raw_feature_spec(schema)
  raw_feature_spec.pop(_LABEL_KEY)

  raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
      raw_feature_spec, default_batch_size=None)
  serving_input_receiver = raw_input_fn()

  transformed_features = tf_transform_graph.transform_raw_features(
      serving_input_receiver.features)

  return tf.estimator.export.ServingInputReceiver(
      transformed_features, serving_input_receiver.receiver_tensors)


def _eval_input_receiver_fn(tf_transform_graph, schema):
  """Build everything needed for the tf-model-analysis to run the model.
  Args:
    tf_transform_graph: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    EvalInputReceiver function, which contains:

      - Tensorflow graph which parses raw untransformed features, applies the
        tf-transform preprocessing operators.
      - Set of raw, untransformed features.
      - Label against which predictions will be compared.
  """
  # Notice that the inputs are raw features, not transformed features here.
  raw_feature_spec = _get_raw_feature_spec(schema)

  serialized_tf_example = tf.compat.v1.placeholder(
      dtype=tf.string, shape=[None], name='input_example_tensor')

  # Add a parse_example operator to the tensorflow graph, which will parse
  # raw, untransformed, tf examples.
  features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)

  # Now that we have our raw examples, process them through the tf-transform
  # function computed during the preprocessing step.
  transformed_features = tf_transform_graph.transform_raw_features(
      features)

  # The key name MUST be 'examples'.
  receiver_tensors = {'examples': serialized_tf_example}

  # NOTE: Model is driven by transformed features (since training works on the
  # materialized output of TFT, but slicing will happen on raw features.
  features.update(transformed_features)

  return tfma.export.EvalInputReceiver(
      features=features,
      receiver_tensors=receiver_tensors,
      labels=transformed_features[_transformed_name(_LABEL_KEY)])


def _input_fn(file_pattern, data_accessor, tf_transform_output, batch_size=200):
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


# TFX will call this function
def trainer_fn(trainer_fn_args, schema):
  """Build the estimator using the high level API.
  Args:
    trainer_fn_args: Holds args used to train the model as name/value pairs.
    schema: Holds the schema of the training examples.
  Returns:
    A dict of the following:

      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  
  train_batch_size = 40
  eval_batch_size = 40

  tf_transform_graph = tft.TFTransformOutput(trainer_fn_args.transform_output)

  train_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      trainer_fn_args.train_files,
      trainer_fn_args.data_accessor,
      tf_transform_graph,
      batch_size=train_batch_size)

  eval_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      trainer_fn_args.eval_files,
      trainer_fn_args.data_accessor,
      tf_transform_graph,
      batch_size=eval_batch_size)

  train_spec = tf.estimator.TrainSpec(  # pylint: disable=g-long-lambda
      train_input_fn,
      max_steps=trainer_fn_args.train_steps)

  serving_receiver_fn = lambda: _example_serving_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_graph, schema)

  exporter = tf.estimator.FinalExporter('forest-fires', serving_receiver_fn)
  eval_spec = tf.estimator.EvalSpec(
      eval_input_fn,
      steps=trainer_fn_args.eval_steps,
      exporters=[exporter],
      name='forest-fires-eval')

  run_config = tf.estimator.RunConfig(
      save_checkpoints_steps=249, keep_checkpoint_max=1)

  run_config = run_config.replace(model_dir=trainer_fn_args.serving_model_dir)

  estimator = _build_estimator(
      # Construct layers sizes with exponetial decay
      hidden_units=[64, 32],
      config=run_config)

  # Create an input receiver for TFMA processing
  receiver_fn = lambda: _eval_input_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_graph, schema)

  return {
      'estimator': estimator,
      'train_spec': train_spec,
      'eval_spec': eval_spec,
      'eval_input_receiver_fn': receiver_fn
  }


Overwriting forest_fires_trainer.py


In [34]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath("forest_fires_trainer.py"),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=500))
context.run(trainer)


INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:/home/thushv89/code/manning_tf2_in_action/Ch14/forest_fires_trainer.py is already loaded, reloading


INFO:tensorflow:Using config: {'_model_dir': 'pipeline/examples/forest_fires_pipeline/Trainer/model_run/28/Format-Serving', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 249, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'pipeline/examples/forest_fires_pipeline/Trainer/model_run/28/Format-Serving', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 249, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:absl:Training model.


INFO:tensorflow:Not using Distribute Coordinator.


INFO:tensorflow:Not using Distribute Coordinator.


...

INFO:tensorflow:Assets written to: pipeline/examples/forest_fires_pipeline/Trainer/model_run/28/Format-TFMA/temp-1623663017/assets


INFO:tensorflow:Assets written to: pipeline/examples/forest_fires_pipeline/Trainer/model_run/28/Format-TFMA/temp-1623663017/assets


INFO:tensorflow:SavedModel written to: pipeline/examples/forest_fires_pipeline/Trainer/model_run/28/Format-TFMA/temp-1623663017/saved_model.pb


INFO:tensorflow:SavedModel written to: pipeline/examples/forest_fires_pipeline/Trainer/model_run/28/Format-TFMA/temp-1623663017/saved_model.pb
INFO:absl:Exported eval_savedmodel to pipeline/examples/forest_fires_pipeline/Trainer/model_run/28/Format-TFMA.
INFO:absl:Serving model copied to: pipeline/examples/forest_fires_pipeline/Trainer/model/28/Format-Serving.
INFO:absl:Eval model copied to: pipeline/examples/forest_fires_pipeline/Trainer/model/28/Format-TFMA.
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 28
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 48
        type_id: 14
        uri: "pipeline/examples/forest_fires_pipeline/Trainer/model/28"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 49
        type_id: 15
        uri: "pipeline/examples/forest_fires_pipeline/Trainer/model_run/28"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))